In [1]:

import hashlib
import time


In [2]:
class service():
    users = {} # format username:[encrypted_pass_word, (set of roles)]
    roles = {}# format rolename: (set of users)
    auth_token = {}
    

    def create_user(self,username,password):
        if username in self.users.keys():
            print('Error: Username already existed')
        else:
            self.users[username] =[hashlib.sha224(password.encode('utf-8')).hexdigest(),set()]
        
    def delete_user(self,username):
        if username in self.users.keys():
            # delete role-user pairs associated with this user
            roles_user =  (self.users[username])[1] #roles associated with this user
            for role in roles_user:
                self.roles[role].remove(username)
            
            #delete user
            del self.users[username]
        else:
            print('Error: Username not existed')
            
            
    def create_role(self,rolename):
        if rolename in self.roles.keys():
            print('Error: Rolename already existed') 
        else:
            self.roles[rolename] = set()
            
    def delete_role(self,rolename):
        if rolename in self.roles.keys():
            #deleter role-user pairs associated with this role
            users_role = self.roles[rolename]#users associated with this role
            for user in users_role:
                (self.users[username])[1].remove(rolename)
            
            #delete role
            del self.roles[rolename]
        else:
             print('Error: Rolename not existed') 
                
    def add_role_to_user(self,username,rolename):
        if rolename not in self.roles.keys():
            print('Error: Rolename not existed') 
            return 
        if username not in self.users.keys():
            print('Error: Username not existed') 
            return 
        if username in self.roles[rolename]:#user already associated with this role
            return
        if rolename in (self.users[username])[1]: #role already associated with this user
            return
        
        # user is not associated with this role, then add
        
        self.roles[rolename].add(username)
        ((self.users[username])[1]).add(rolename)
        
    def authenticate(self,username,password):
        en_pass = hashlib.sha224(password.encode('utf-8')).hexdigest() #encrypted_pass_word
        expire = 7200 # expire time in senconds, 7200s = 2h
        
        if username not in self.users.keys():
            print('Error: Username not existed')
            return 
        if en_pass != (self.users[username])[0]:
            print('Error: password not correct')
            return
        # username and password correct, then produce token
        ts_str = str(time.time() + expire)
        token = ts_str+':'+ en_pass
        self.auth_token[token] = username
        return token
        
    def invalidate(self,token):
        if token not in self.auth_token.keys():
            print('Error: Token not existed')
            return 
        del self.auth_token[token]
        
    def check_role(self,token,role):
        # check the token
        if token not in self.auth_token.keys():
            print('Error: Token not existed')
            return 'Error'
        
        tokenlist = token.split(':') # split the expire time and token generated
        if len(tokenlist)>2:        # format not valid
            print('Error: Token not valid')
            return 'Error'
        
        exp_time = tokenlist[0] # expire time
        if float(exp_time) < time.time(): # expired
            print('Error: Token expired')
            self.invalidate(token)
            return 'Error'
        
        #check role 
        username = self.auth_token[token] # get username, in order to get the set of roles associated with thi user
        if role in (self.users[username])[1]:
            return True
        else:
            return False
        
        
    def all_role(self,token): 
        #check the token
        if token not in self.auth_token.keys():
            print('Error: Token not existed')
            return 'Error'
        
        tokenlist = token.split(':') # split the expire time and token generated
        if len(tokenlist)>2:        # format not valid
            print('Error: Token not valid')
            return 'Error'
        
        exp_time = tokenlist[0] # expire time
        if float(exp_time) < time.time(): # expired
            print('Error: Token expired')
            self.invalidate(token)
            return 'Error'
        
        # get roles 
        username = self.auth_token[token]# get username, in order to get the set of roles associated with thi user
        roles_user = (self.users[username])[1] #roles of this user
        return roles_user
        
        
        
        
        
        
    

# TEST

In [3]:
testcase = service()

## Test create_user()

In [4]:
testcase.create_user('aa','123')

In [5]:
testcase.users

{'aa': ['78d8045d684abd2eece923758f3cd781489df3a48e1278982466017f', set()]}

## Test delete_user()

In [6]:
testcase.delete_user('bb')

Error: Username not existed


In [7]:
testcase.delete_user('aa')

In [8]:
testcase.users

{}

## Test create_role()

In [9]:
testcase.create_user('aa','123')

In [10]:
testcase.create_role('admin')

In [11]:
testcase.create_role('admin')

Error: Rolename already existed


In [12]:
testcase.roles

{'admin': set()}

## Test delete_role()

In [13]:
testcase.delete_role('wrongname')

Error: Rolename not existed


In [14]:
testcase.delete_role('admin')
testcase.roles

{}

## Test add_role_to_user()

In [15]:
testcase.create_role('admin')
testcase.add_role_to_user('wrong_user_name','admin')

Error: Username not existed


In [16]:
testcase.add_role_to_user('aa','wrongname')

Error: Rolename not existed


In [17]:
testcase.add_role_to_user('aa','admin')

In [18]:
testcase.users

{'aa': ['78d8045d684abd2eece923758f3cd781489df3a48e1278982466017f', {'admin'}]}

In [19]:
testcase.roles

{'admin': {'aa'}}

## Test authenticate()

In [20]:
testcase.authenticate('wrong_username','wrong_password')

Error: Username not existed


In [21]:
testcase.authenticate('aa','wrong_password')

Error: password not correct


In [22]:
token = testcase.authenticate('aa','123')
token

'1653124614.649365:78d8045d684abd2eece923758f3cd781489df3a48e1278982466017f'

## Test invalidate()

In [23]:
testcase.invalidate('wrong_token')

Error: Token not existed


In [24]:
testcase.invalidate(token) 

In [25]:
testcase.invalidate(token)

Error: Token not existed


In [26]:
token = testcase.authenticate('aa','123')
token

'1653124614.7096968:78d8045d684abd2eece923758f3cd781489df3a48e1278982466017f'

## Test check_role()

In [27]:
testcase.check_role('wrong_token','wrong_rolename')

Error: Token not existed


'Error'

In [28]:
testcase.check_role(token,'wrong_rolename')

False

In [29]:
testcase.check_role(token,'admin')

True

## Test the expiration of token

In [30]:
token_list = token.split(':')
exp_time = float(token_list[0])
exp_time=exp_time-7200 #expried time
exp_token = str(exp_time)+':'+ token_list[1]
testcase.auth_token[exp_token ] = 'aa'  #add expried time to the tokens

testcase.check_role(exp_token,'admin')

Error: Token expired


'Error'

## Test all_role()

In [31]:
testcase.all_role('wrong_token')

Error: Token not existed


'Error'

In [32]:
testcase.all_role(token)

{'admin'}